In [1]:
from matplotlib import pyplot as plt
import tensorflow as tf
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torchvision import transforms as T
from torchvision import transforms
import torch
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import pytorch_msssim
import torchvision.models as models
import torchvision.transforms as transforms
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

!nvidia-smi

2024-08-12 14:05:54.396065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 14:05:54.408738: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 14:05:54.413160: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-12 14:05:54.424293: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-12 14:05:55.109385: W tensorflow/compiler/tf2

CUDA is available!  Training on GPU ...
cuda
Mon Aug 12 14:05:57 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.67                 Driver Version: 550.67         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060        Off |   00000000:01:00.0  On |                  N/A |
|  0%   52C    P2             N/A /  115W |    2391MiB /   8188MiB |     33%      Default |
|                                         |                        |                  N/A |
+--

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

HIDDEN_SIZE = 256
xx = HIDDEN_SIZE//8
embedding = 128


# Определение модели энкодера
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, HIDDEN_SIZE, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(HIDDEN_SIZE, HIDDEN_SIZE, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(HIDDEN_SIZE, HIDDEN_SIZE//2, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(HIDDEN_SIZE//2 * xx * xx, embedding*4)
        self.fc2 = nn.Linear(embedding*4, embedding*2)
        self.fc3 = nn.Linear(embedding*2, embedding)
        self.drop = nn.Dropout(0.25)

    def forward(self, x):
        x = torch.tanh(self.conv1(x))
        x = self.pool(x)
        x = torch.tanh(self.conv2(x))
        x = self.pool(x)
        x = self.drop(x)
        x = torch.tanh(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc1(x)
        x = self.drop(x)
        x = self.fc2(x)
        coords = self.fc3(x)
        return coords

# Определение модели декодера
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(embedding, embedding*2)
        self.fc2 = nn.Linear(embedding*2, embedding*4)
        self.fc3 = nn.Linear(embedding*4, HIDDEN_SIZE//2 * xx * xx)
        self.deconv1 = nn.ConvTranspose2d(HIDDEN_SIZE//2, HIDDEN_SIZE, kernel_size=3, padding=1)
        self.deconv2 = nn.ConvTranspose2d(HIDDEN_SIZE, HIDDEN_SIZE, kernel_size=3, padding=1)
        self.up = nn.Upsample(scale_factor=2, mode='nearest')
        self.deconv3 = nn.ConvTranspose2d(HIDDEN_SIZE, 3, kernel_size=3, padding=1)
        self.drop = nn.Dropout(0.25)

    def forward(self, coords):
        x = self.fc1(coords)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.fc3(x)
        x = x.view(x.size(0), HIDDEN_SIZE//2, xx, xx)  # Reshape to match deconv1's input
        x = self.up(x)
        x = torch.tanh(self.deconv1(x))
        x = self.drop(x)
        x = self.up(x)
        x = torch.tanh(self.deconv2(x))
        x = self.up(x)
        decoded = torch.tanh(self.deconv3(x))
        return decoded

# Полный автоэнкодер
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        coords = self.encoder(x)
        decoded = self.decoder(coords)
        return decoded, coords

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Autoencoder().to(device)


In [4]:
# Load модели
model_path = 'autoencoder_model_test.h5'
model = Autoencoder()
model.load_state_dict(torch.load(model_path))
model.eval()

/tmp/ipykernel_38846/1136507937.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Autoencoder(
  (encoder): Encoder(
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fc1): Linear(in_features=131072, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=256, bias=True)
    (fc3): Linear(in_features=256, out_features=128, bias=True)
    (drop): Dropout(p=0.25, inplace=False)
  )
  (decoder): Decoder(
    (fc1): Linear(in_features=128, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=512, bias=True)
    (fc3): Linear(in_features=512, out_features=131072, bias=True)
    (deconv1): ConvTranspose2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (deconv2): ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [ ]:
HIDDEN_SIZE = 256
resize_to = HIDDEN_SIZE

image_path = "./map/test/tiles/cell_128_128.png"
test_img_1 = Image.open(image_path).convert("RGB")
test_img_1 = test_img_1.resize((resize_to, resize_to))



with torch.no_grad():
    outputs, _ = model(noisy_inputs)
    outputs = outputs.cpu().numpy()